## Linear Regression as a model of Emotional Appraisal

The first example we discuss involves building a simple linear regression model in Pyro. We consider a case that is relevant to affective computing --- how should we build a model to reason about someone's emotions? There are lots of emotion theories that one can draw from, and probabilistic programming offers an elegant way to specify (and test!) these theories.


Before we jump in, we want to introduce the dataset that we'll be working with throughout the rest of the tutorial. This dataset is of a managable size (so training shouldn't take too long), but still complex enough that we can demonstrate some of the nice features of applying probabilistic programming to model real-life, AI-relevant examples (in this case, relevant to affective computing).

### Dataset

We will be using data from a published paper (Ong, Zaki, & Goodman, 2015; Experiment 3), which is available [here](https://github.com/desmond-ong/affCog) but we have also reproduced the data in the current repository. In this experiment, we showed human participants an agent playing a gamble; he spins a wheel with three possible outcomes, and wins the amount on the wheel. 

<div style="width: 500px; margin: auto;">![Experiment Summary](images/experimentSummary.png)</div>
(Figure modified from Ong et al, 2015, Figure 9).

On some trials, participants see the outcome that the agent won ((i) above). On other trials ((ii) above), participants were not shown the outcome, but instead were shown what ostensibly was the agent's facial expression after seeing the outcome. And on the last third of trials ((iii) above), participants were shown both the outcome and the agent's facial expression.
Following these, participants were asked to rate how they thought the agent felt, on 8 emotions, using a 9 point Likert scale.

Thus, the dataset consists of some "outcome only" trials where participants saw outcomes and rated the agent's emotions, "facial expression only" trials where participants attributed emotions to a facial expression, and trials where they saw both and had to integrate the information from both the outcome and the facial expression to make a judgment.

### Appraising the outcome

Let us first consider the "outcome only" trials. Many established emotion theories and affective computing theories hold that people experiencing events (e.g., winning the lottery, missing the bus) will evaluate the situation according to a set of important features. Was the outcome desirable? Was the outcome surprising? Was the outcome controllable? This evaluation is known as **appraisal**.

Put another way, the number of situations that people encounter in daily life vary immensely along a large number of dimensions, some important (the amount that one wins in the lottery) and some not so important (the color of the lottery ticket). **Appraisal** is computationally necessary to reduce the complexities of everyday situations into a low dimensional set of emotion-relevant dimensions.


### A Linear Regression model with Appraisal

Thus, we already have the few basic ingreidents of our theory. We have an observable variable (the outcome). We have an appraisal process that converts the outcome into a small number of relevant features. And we have the emotion ratings that people produce. Let's construct a basic regression model:

<div style="width: 300px; margin: auto;">![Graphical Model](images/graphicalModel_LinearRegression.png)</div>

Here's how to read the model above, which uses graphical model notation. Shaded circles represent observed variables, while unshaded represents latent, or unobserved variables. Small rectangles represent parameters (to be fitted). We have $N$ i.i.d. pairs of (*Outcome*, *Emotion Ratings*), where $N$ is the size of the dataset, and so these are represented in the large rectangle (called a "plate"), to indicate that they are repeated $N$ times. And between these *Outcome* and *Rating* pairs, we have an appraisal transformation.

In a linear regression, we have $K+1$ regression weights that map the appraisal to the emotion ratings (the $+1$ is for the bias term). If the appraisal variables are given by $\{1, a_1, a_2, \ldots, a_K\} = \vec{a}$, we can write a regression equation: 

*Rating* = $\vec{\beta} \cdot \vec{a}$ = $\beta_0$ + $\beta_1 a_1$ + $\beta_2 a_2$ + $\ldots$ + $\beta_K a_K$ + $\epsilon$

where the $\beta_i$'s represent the regression weights for the $i$-th appraisal variable, $\beta_0$ is the bias term and $\epsilon$ is an error term. Notice that the $\beta_i$'s should remain the same across all $N$ observations: thus, it is left out of the "plate" in the model diagram above.

We assume that each $\beta_i$ is drawn from a Normal distribution parameterized by a mean (location parameter) $\mu_i$ and a standard deviation (or scale parameter) $\sigma_i$, i.e., $\beta_i \sim N(\mu_i, \sigma_i)$. We wish to learn these parameters $\mu_i, \sigma_i$ from the data.


(Note: *Appraisal* in this model is a strange creature. We could represent it as a latent variable (so like *outcome*, but unshaded). Here, to reflect the fact that *appraisal* is a modular function that can be tested scientifically against data, we chose to represent it more like a fittable parameter.).

Let's write some Pyro!

##### Preamble

This first chunk of code imports the necessary python packages and functions that we will use

In [ ]:
from __future__ import division, print_function, absolute_import

%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import pyro
import pyro.distributions as dist
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

This next chunk defines some variable and names that are specific to this dataset, as well as a function to read in the data.

The data is stored in `outcome_emotion_dataset`, which is a torch Tensor of size (1541, 17), indicating that there are N=1,541 observations of 17 variables. The first 9 are the parameterization of the outcome (the 3 payoffs on the wheel and their probabilities, which outcome they won and that probability, and the angle within the sector that the wheel landed on), and the next 8 are the emotion variables. All the variables are scaled so that they lie within [0,1].

In [ ]:
# data location
dataset_path = os.path.join(os.path.abspath('..'), "CognitionData", "data_wheelOnly.csv")

OUTCOME_VAR_NAMES = ["payoff1", "payoff2", "payoff3", "prob1", "prob2", "prob3", "win", "winProb", "angleProp"]
EMOTION_VAR_NAMES = ["happy", "sad", "anger", "surprise", "disgust", "fear", "content", "disapp"]
OUTCOME_VAR_DIM = len(OUTCOME_VAR_NAMES)
EMOTION_VAR_DIM = len(EMOTION_VAR_NAMES)

def load_outcome_emotion_dataset(csv_file, normalize_values=True, preview_datafile=False):
    data_readin = pd.read_csv(csv_file)
    outcome_data = data_readin.loc[:,OUTCOME_VAR_NAMES]
    if normalize_values:
        ####
        ## payoff1, payoff2, payoff3 and win are between 0 and 100
        ## need to normalize to [0,1] to match the rest of the variables,
        ## by dividing payoff1, payoff2, payoff3 and win by 100
        ####
        outcome_data.loc[:,"payoff1"] = outcome_data.loc[:,"payoff1"]/100
        outcome_data.loc[:,"payoff2"] = outcome_data.loc[:,"payoff2"]/100
        outcome_data.loc[:,"payoff3"] = outcome_data.loc[:,"payoff3"]/100
        outcome_data.loc[:,"win"]     = outcome_data.loc[:,"win"]/100
    outcome_data_tensor = torch.tensor(outcome_data.values).type(torch.Tensor)
    
    # the actual data has 8 emotions, but for illustration we just use 1 emotion, happy
    # the rest of the functions below assume a 1-D "y" variable
    emotion_data = data_readin.loc[:,EMOTION_VAR_NAMES]
    #emotion_data = data_readin.loc[:, "happy"]
    if normalize_values:
        ## note that emotions are transformed from a 9 point Likert to [0,1] via emo <- (emo-1)/8
        emotion_data   = (emotion_data-1)/8
    #emotion_data = emotion_data.values.reshape( emotion_data.shape[0] , 1)
    #emotion_data = torch.tensor(emotion_data).type(torch.Tensor)
    emotion_data_tensor = torch.tensor(emotion_data.values).type(torch.Tensor)
    
    if preview_datafile:
        print("Preview of first 3 rows:")
        print(outcome_data.loc[0:2,:])
        print(emotion_data.loc[0:2,:])
    
    data = torch.cat((outcome_data_tensor, emotion_data_tensor), 1)
    return data


# reads in datafile.
print("Reading in dataset...")
outcome_emotion_dataset = load_outcome_emotion_dataset(csv_file=dataset_path, preview_datafile=True)
N_samples = outcome_emotion_dataset.shape[0]

print("Shape of dataset: ", outcome_emotion_dataset.shape)

`compute_appraisal()` is a function that takes in an outcome vector, and returns a vector of appraisal values. The example below reproduces the appraisal function used in Ong et al (2015). But more generally, this is a modular function that can be substituted out to test other possible operationalizations of appraisal theories.

In [ ]:
def compute_appraisal(outcome_data):
    # We have a simple hard-coded these appraisals, for illustration
    # This is following Ong, Zaki, & Goodman (2015)
    # the outcome data columns are, in order:
    # ["payoff1", "payoff2", "payoff3", "prob1", "prob2", "prob3", "win", "winProb", "angleProp"]
    # the 3 appraisal variables are: 
    #     amount won ("win"),
    #     Prediction Error PE = win - EV, where EV = prob1*payoff1 + prob2*payoff2 + prob3*payoff3
    #     absolute value of PE
    
    # if outcome_data only has 1 observation, reshape so vectorization works
    if(len(outcome_data.shape)==1):
        outcome_data = outcome_data.view(1,9)
        print(outcome_data.shape)
    
    # initializing appraisalVals
    appraisalVals = torch.zeros(size=(outcome_data.shape[0],3))
    appraisalVals[:,0] = outcome_data[:,6] # amount won
    
    # Expected value
    EV = outcome_data[:,0] * outcome_data[:,3] + \
         outcome_data[:,1] * outcome_data[:,4] + \
         outcome_data[:,2] * outcome_data[:,5]
    
    # prediction error and absolute PE
    appraisalVals[:,1] = appraisalVals[:,0] - EV
    appraisalVals[:,2] = abs(appraisalVals[:,1])
    return(appraisalVals)


##### Model

Next, we have the model. Let's break down what goes on.

First, the model samples some $\beta$ coefficients from a Normal with some priors over the mean and the scale (in this case, mean of 0 and scale of 1).
This is achieved using the `pyro.sample()` function. 
For example,

`b_0 = pyro.sample("b_0", Normal(coeff_mean_prior, coeff_scale_prior))`


Note that the `sample()` function takes in a variable name, which allows Pyro to uniquely identify that variable in its variable store. (As such, the variable names are unique, and you can only have one `sample()` function with a particular variable name in this function).



Next, the function will loop over the observed data, using `pyro.iarange()`. This function defines a special Pyro environment with a unique name (`"map"`), within which Pyro understands that each iteration of the "loop" is conditionally independent. Thus, the computation on each data-point is conditionally independent from the computation on other data-points. (This reflects the plate-notation in the model above; each datapoint is independent, BUT the $\beta$ coefficients are the same across all of them, that's why they were defined before the `pyro.iarange()` loop)

Within this loop, we take the `outcome_data`, run it through `compute_appraisal()` to get a small 3-dimensional `appraisal_vars`. We manually compute the regression equation:

`prediction = b_0 + b_1 * appraisal_vars[:,0] + b_2 * appraisal_vars[:,1] + b_3 * appraisal_vars[:,2]`

Thus, `prediction` is the mean of the Normal distribution that the linear regression model predicts.
Finally, we condition on the observed data:

`pyro.sample("obs", Normal(prediction, 1), obs = emotion_data)`

Notice we use `pyro.sample()` again. We draw a sample from a Normal with mean `prediction` and scale 1, but this time, we condition that this sample is equal to the observed `emotion_data`, using the argument `obs = ...`.


And that's basically it for this function. Pyro's `irange()` and `iarange()` [functions](http://pyro.ai/examples/svi_part_ii.html#iarange) allow a flexible way to perform computations on individual datapoints while taking care of conditional independencies (the `i` in `irange()` and `iarange()`). One difference is that `iarange()` is vectorized, so we can perform the calculations on the entire data Tensor instead of individual observations.



In [ ]:
def fit_regression_model(data):
    # define the parameters that control the gaussian prior over the regression coeffs.
    # mean = 0, scale = 1
    coeff_mean_prior = torch.tensor(0.0)
    coeff_scale_prior = torch.tensor(1.0)
    
    # sample b_0 (intercept) and b_1 to b_3 (regression coeffs)
    b_0 = pyro.sample("b_0", Normal(coeff_mean_prior, coeff_scale_prior))
    b_1 = pyro.sample("b_1", Normal(coeff_mean_prior, coeff_scale_prior))
    b_2 = pyro.sample("b_2", Normal(coeff_mean_prior, coeff_scale_prior))
    b_3 = pyro.sample("b_3", Normal(coeff_mean_prior, coeff_scale_prior))
    
    # loop over observed data
    with pyro.iarange("map", data.shape[0]):
        outcome_data = data[:, :(OUTCOME_VAR_DIM)]
        # Here, for simplification, we are only taking one emotion variable (happy)
        # instead of all 8 emotions
        emotion_data = data[:, OUTCOME_VAR_DIM]  
        appraisal_vars = compute_appraisal(outcome_data)
        
        # run the regression forward
        prediction = b_0 + b_1 * appraisal_vars[:,0] + b_2 * appraisal_vars[:,1] + b_3 * appraisal_vars[:,2]
        # condition on the observed data
        pyro.sample("obs", Normal(prediction, 1), obs = emotion_data)

##### Guide

We will use stochastic variational inference to do inference  (e.g., see [here](http://pyro.ai/examples/svi_part_i.html) and [here](http://pyro.ai/examples/svi_part_ii.html), or next example for more description). SVI involves using a parameterized distribution to help approximate the posterior distribution that we want to infer. Practically this means defining a function that runs 'parallel' to the `model()` that helps to guide the `model()` in making better sampling choices. Hence, this function is sometimes called the `guide()`. 

Specifically, we define the `guide()` such that:

- it contains variational parameters via `pyro.param()`. In this case, these parameters are the $\mu_i, \sigma_i$'s that parameterize the regression parameters $b_i \sim N(\mu_i, \sigma_i)$. 
- it has the same unconditional `pyro.sample()` call signature as `model()`. In this case, `pyro.sample("b_0",...)` through `"b_4"`
- it does not have any conditioned `pyro.sample()` calls.

In [ ]:
def fit_regression_guide(data):
    mean_b0_param  = pyro.param("guide_mean_b0",  torch.tensor(0.0))
    scale_b0_param = pyro.param("guide_scale_b0", torch.tensor(1.0))
    mean_b1_param  = pyro.param("guide_mean_b1",  torch.tensor(0.0))
    scale_b1_param = pyro.param("guide_scale_b1", torch.tensor(1.0))
    mean_b2_param  = pyro.param("guide_mean_b2",  torch.tensor(0.0))
    scale_b2_param = pyro.param("guide_scale_b2", torch.tensor(1.0))
    mean_b3_param  = pyro.param("guide_mean_b3",  torch.tensor(0.0))
    scale_b3_param = pyro.param("guide_scale_b3", torch.tensor(1.0))
    # sample coefficients from Normal(mean, scale)
    pyro.sample("b_0", Normal(mean_b0_param, scale_b0_param))
    pyro.sample("b_1", Normal(mean_b1_param, scale_b1_param))
    pyro.sample("b_2", Normal(mean_b2_param, scale_b2_param))
    pyro.sample("b_3", Normal(mean_b3_param, scale_b3_param))
    

##### Fitting the model

Next, we can proceed to actually fit the model. The first step is to refresh the parameter store using `pyro.clear_param_store()`.

We will use Stochastic Variational Inference `SVI()` which takes in the model and guide that we wrote above, as well as an optimization algorithm (here we use `torch.optim.Adam()`) and a loss function (here we use `Trace_ELBO()`). When `svi.step(data)` is called, it runs SVI over the `data`. Thus, here we simply define a loop that runs over the entire dataset `num_iterations` times. (We can easily modify this to do mini-batching, for example, for large datasets.)

In [ ]:
pyro.clear_param_store()

num_iterations = 1000

# setup the optimizer with some learning rate
optimizer = Adam({"lr": 0.005})

# setup the inference algorithm
svi = SVI(fit_regression_model, fit_regression_guide, optimizer, loss=Trace_ELBO())

# do gradient steps
losses = []
for thisIteration in range(num_iterations):
    # calculate the loss and take a gradient step
    thisLoss = svi.step(outcome_emotion_dataset)
    losses.append(thisLoss)
    if thisIteration % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (thisIteration + 1, thisLoss / float(N_samples)))

plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss")

In [ ]:
# output the learned variational parameters
print("b0 ~ Normal(%.4f, %.4f)" % (pyro.param("guide_mean_b0").item(), pyro.param("scale_b0_param").item()))
print("b1 ~ Normal(%.4f, %.4f)" % (pyro.param("guide_mean_b1").item(), pyro.param("scale_b1_param").item()))
print("b2 ~ Normal(%.4f, %.4f)" % (pyro.param("guide_mean_b2").item(), pyro.param("scale_b2_param").item()))
print("b3 ~ Normal(%.4f, %.4f)" % (pyro.param("guide_mean_b3").item(), pyro.param("scale_b3_param").item()))

### Coda

Hopefully the example above illustrates several appealing features of probabilistic programming:

- We can use PPLs to specify theory, and importantly, uncertainty in theory and random processes.
- Theory is represented as modular chunks of code. (As in our `compute_appraisal()` function, or in the linear regression model.) This allows us to substitute out different parts of the model, which is handy for optimization or theory testing!
    - Want to try a different appraisal representation? No problem! 
    - Want to try using a feed-forward neural network instead of a linear regression? Sure (and in fact we will cover that in the next example!)
- Inference and learning is orthogonal to model specification. Thus, the modeler can focus on specifying the model, while Pyro does most of the heavy lifting (by leveraging PyTorch modules).


In the next example, we show how to incorporate a feed-forward neural network in order to build a deep generative model.


------

##### Extra: Some further extensions to the linear regression model

In the following code we modify the earlier model to handle an arbitrary number of regression parameters, and introduce the notion of "lifting" models to get distributions over *models* that we can sample from.

Instead of using a low-dimensional `compute_appraisal()` function, we demonstrate a standard linear regression by defining an `appraisalRegressionModule()` that inherits from PyTorch's `nn.module`. (Note that a linear regression can be thought of as a feed-forward neural network with 0 hidden layers and no non-linearities, such that the output layer is just a linear combination of the input units). This is achieved using `nn.Linear()`:

In [ ]:
class appraisalRegressionModule(nn.Module):
    def __init__(self, num_features):
        super(appraisalRegressionModule, self).__init__()
        self.linear = nn.Linear(num_features, 1)

    def forward(self, outcome):
        return self.linear(outcome)

regression_model = appraisalRegressionModule(OUTCOME_VAR_DIM)

Next, we write the model. The main difference now is that we can define separate location and scale priors for each regressor, given by `weights_loc` and `weights_scale` below, which are each Tensors of Size(1,`OUTCOME_VAR_DIM`) (or any arbitrary length given by an appraisal transformation).

Rather than typing out `OUTCOME_VAR_DIM` `pyro.sample()` statements, a fancier way of sampling using `weights_loc, weights_scale` is to "lift" the regression model using `pyro.random_module()`. This takes a model and a prior, and returns a *distribution over possible models*. In other words, we can think of this step as sampling models from a "model-distribution" (which under the hood, Pyro does by sampling the regression weights from their priors). Note that we do not see any `pyro.sample()` calls in the code below, as compared to the earlier code: these are taken care of by Pyro and will be executed when sampling from the distribution returned by `random_module()`.

The rest of the code is the same as above.

In [ ]:
def bayesianRegressionModel(data):
    # Create unit normal priors over the parameters
    weights_loc   = torch.zeros(size=(torch.Size((1, OUTCOME_VAR_DIM))))
    weights_scale = torch.ones(size=(torch.Size((1, OUTCOME_VAR_DIM))))
    weights_prior = Normal(weights_loc, weights_scale).independent(1)
    
    # location and scale prior for the bias
    bias_loc   = torch.zeros(size=(torch.Size((1, ))))
    bias_scale = torch.ones(size=(torch.Size((1, ))))
    bias_prior = Normal(bias_loc, bias_scale).independent(1)
    
    priors = {'linear.weight': weights_prior, 'linear.bias': bias_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a model (which also samples from weights_prior and bias_prior)
    sampled_regression_model = lifted_module()
    
    with pyro.iarange("map", data.shape[0]):
        outcome_data = data[:, :(OUTCOME_VAR_DIM)]
        # Here, for simplification, we are only taking one emotion variable (happy)
        # instead of all 8 emotions
        emotion_data = data[:, OUTCOME_VAR_DIM]  
        
        # run the regressor forward conditioned on data
        prediction = sampled_regression_model(outcome_data).squeeze(-1)
        # condition on the observed data
        pyro.sample("obs", Normal(prediction, 1), obs = emotion_data)

We write the guide function in a parallel manner to the model function. Note that we register the variational parameters in Pyro's parameter store using the `pyro.param()` function, just like above. We do the same trick of lifting the regression model using a `pyro.random_module()` call and sample from it.

In [ ]:
def bayesianRegressionGuide(data):
    # define our variational parameters
    weights_loc   = torch.randn(1, OUTCOME_VAR_DIM)
    # Note that the scale has to be non-negative. Thus, we use exp() to get a non-negative number.
    # we also use a narrower scale (exp(-1) ~ 0.35 instead of exp(0) = 1)
    weights_scale = torch.exp(-1.0 * torch.ones(1, OUTCOME_VAR_DIM) + 0.05 * torch.randn(1, OUTCOME_VAR_DIM))
    bias_loc      = torch.randn(1)
    bias_scale    = torch.exp(-1.0 * torch.ones(1) + 0.05 * torch.randn(1))

    # using pyro.param() to register the variational parameters
    weight_loc_param   = pyro.param("guide_loc_weight", weights_loc)
    weight_scale_param = pyro.param("guide_scale_weight", weights_scale)
    bias_loc_param     = pyro.param("guide_loc_bias", bias_loc)
    bias_scale_param   = pyro.param("guide_scale_bias", bias_scale)
    # guide distributions for w and b
    weight_dist = Normal(weight_loc_param, weight_scale_param).independent(1)
    bias_dist   = Normal(bias_loc_param, bias_scale_param).independent(1)
    dists = {'linear.weight': weight_dist, 'linear.bias': bias_dist}
    # lift the module and sample from that distribution
    lifted_module = pyro.random_module("module", regression_model, dists)
    return lifted_module()

Next we fit this regression model using SVI; this is identical to the code earlier.

In [ ]:
pyro.clear_param_store()

num_iterations = 1000

# setup the optimizer with some learning rate
optimizer = Adam({"lr": 0.005})

# setup the inference algorithm
bayesianRegressionSVI = SVI(bayesianRegressionModel, bayesianRegressionGuide, optimizer, loss=Trace_ELBO())

# do gradient steps
losses = []
for thisIteration in range(num_iterations):
    # calculate the loss and take a gradient step
    thisLoss = bayesianRegressionSVI.step(outcome_emotion_dataset)
    losses.append(thisLoss)
    if thisIteration % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (thisIteration + 1, thisLoss / float(N_samples)))
        

plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss")

In [ ]:
#for name in pyro.get_param_store().get_all_param_names():
#    print(name, pyro.param(name).data.numpy())

guide_loc_weight = pyro.param("guide_loc_weight")[0]
guide_scale_weight = pyro.param("guide_scale_weight")[0]

print("b0 ~ Normal(%.4f, %.4f)" % (pyro.param("guide_loc_bias").item(), pyro.param("guide_scale_bias").item()))
for j in xrange(len(guide_loc_weight)):
    print("b%1d" % (j+1), "~ Normal(%.4f, %.4f)" % (guide_loc_weight[j], guide_scale_weight[j]))
    

-----

Written by: Desmond Ong (desmond.c.ong@gmail.com)

References:

Pyro [Bayesian Regression tutorial](http://pyro.ai/examples/bayesian_regression.html)


Data from https://github.com/desmond-ong/affCog, from the following paper:

Ong, D. C., Zaki, J., & Goodman, N. D. (2015). Affective Cognition: Exploring lay theories of emotion. *Cognition*, 143, 141-162.